In [1]:
import pickle

dictName = f'../../data/T2Dcounts.pkl'
dataMap = None

with open(dictName, 'rb') as f:
    dataMap = pickle.load(f)
    
print(list(dataMap.keys()))
print('Complete')

['T2D', 'NGT', 'T2D_001', 'NGT_003', 'T2D_002', 'NGT_010', 'T2D_006', 'T2D_007', 'NGT_016', 'T2D_009', 'NGT_018', 'T2D_011', 'NGT_020', 'T2D_012', 'NGT_021', 'T2D_017', 'NGT_022', 'T2D_019', 'NGT_023', 'T2D_024', 'NGT_027', 'T2D_026', 'NGT_028', 'T2D_030', 'NGT_033', 'T2D_031', 'NGT_050', 'T2D_035', 'NGT_052', 'T2D_101', 'NGT_123', 'T2D_104', 'NGT_124', 'T2D_107', 'T2D_108', 'NGT_127', 'T2D_109', 'T2D_112', 'NGT_129', 'T2D_119']
Complete


In [2]:
print(dataMap['T2D'])
print(dataMap['NGT'])
print(list(dataMap['T2D_001'].keys()))

{'T2D_107', 'T2D_026', 'T2D_030', 'T2D_011', 'T2D_109', 'T2D_101', 'T2D_119', 'T2D_031', 'T2D_002', 'T2D_108', 'T2D_019', 'T2D_001', 'T2D_104', 'T2D_007', 'T2D_112', 'T2D_024', 'T2D_009', 'T2D_035', 'T2D_017', 'T2D_012', 'T2D_006'}
{'NGT_124', 'NGT_010', 'NGT_003', 'NGT_023', 'NGT_050', 'NGT_123', 'NGT_028', 'NGT_052', 'NGT_020', 'NGT_033', 'NGT_022', 'NGT_127', 'NGT_016', 'NGT_021', 'NGT_027', 'NGT_018', 'NGT_129'}
['basal', 'post', 'rec']


In [3]:
print(dataMap['T2D_001']['basal'].shape)

(66023,)


In [4]:
countName = '../../data/GSE202295_gene_counts.txt'

import re
import numpy as np

pat = re.compile('".*" "(.*)" "(.*)" (.*)')

def parseCountsLine(line, pat):
    res = pat.search(line)
    try:
        groups = res.groups()
        counts = [int(c) for c in groups[2].split(' ')]
        return (groups[0], groups[1], counts)
    except Exception as e:
        return None
    
def parseCountsFirstLine(line):
    parts = line.split(' ')
    pat = re.compile('"([^A]+).*"')
    countsMap = {}
    for i in range(2,len(parts)):
        sampleId = pat.search(parts[i]).groups()[0]
        countsMap[sampleId] = i-2
    return countsMap
    
countsAll = []
countNames1 = []
countNames2 = []
countsMap = None

with open(countName, 'r') as f:
    for line in f.readlines():
        res = parseCountsLine(line, pat)
        if res is None:
            countsMap = parseCountsFirstLine(line)
            continue
        n1,n2,counts = res
        countNames1.append(n1)
        countNames2.append(n2)
        countsAll.append(np.array(counts))
        
countsAll = np.stack(countsAll)
print(countsAll.shape)
print(countNames2[0:10])
print('Complete')

(66023, 111)
['RP11-399E6.2', 'AC093151.1', 'FO681542.1', 'RP11-399E6.4', 'DPPA2P2', 'RPS6KA1', 'MIR1976', 'RN7SL679P', 'Y_RNA', 'RNU7-29P']
Complete


In [65]:
import re

imp = []
with open('../../data/omics-regions/post_Corr.txt', 'r') as f:
    for line in f.readlines():
        imp = [int(num) for num in re.split('\s+', line) if len(num) > 0]
        
imp = np.array(imp)
print(imp)

[43567 26941 52161 59236 23705  3014 12102  8093 33518 18699 60217 29246
 26520 11845 47688  8623 35479 25801  2369 18803]


In [7]:
nRuns = 40
C = 0.001
timep = 'post'
model = f'Logistic Regression L2 C={C}'
desc = f'T2D({timep}) vs NGT({timep}) exercise muscle biopsy genecounts W*Count'

import random
import numpy as np

def getSamples(dataMap, timep):
    ngt = []
    t2d = []
    for sid in dataMap['NGT']:
        if timep in dataMap[sid]:
            ngt.append(dataMap[sid][timep])
    for sid in dataMap['T2D']:
        if timep in dataMap[sid]:
            t2d.append(dataMap[sid][timep])
    return np.stack(ngt), np.stack(t2d)

from sklearn.linear_model import LogisticRegression
import sys

sys.path.append('../../src')

from imagenomer import Analysis, JsonData, JsonSubjects

analysis = Analysis(f'{desc}: {model}', 'localhost', 8000)

a,b = np.triu_indices(264,1)
idcs = np.arange(34716)

labels = [f'{a[i]}-{b[i]}' for i in idcs]

print(labels[0:10])
print(labels[-10:])

tot = []

for run in range(nRuns):
    ngt, t2d = getSamples(dataMap, timep)
    np.random.shuffle(ngt)
    np.random.shuffle(t2d)
    a = int(3*ngt.shape[0]/4)
    b = int(3*t2d.shape[0]/4)
    c = ngt.shape[0]-a
    d = t2d.shape[0]-b
    Xtrain = np.concatenate([ngt[:a], t2d[:b]])
    Xtest = np.concatenate([ngt[a:], t2d[b:]])
    ytrain = np.concatenate([np.zeros(a), np.ones(b)])
    ytest = np.concatenate([np.zeros(c), np.ones(d)])
    
#     mu_Xtrain = np.mean(Xtrain, axis=0, keepdims=True)
#     sd_Xtrain = np.std(Xtrain, axis=0, keepdims=True)
    
#     sd_Xtrain[sd_Xtrain == 0] = 1
    
#     Xtrain = (Xtrain-mu_Xtrain)/sd_Xtrain
#     Xtest = (Xtest-mu_Xtrain)/sd_Xtrain
    
#     print(Xtrain.shape)
#     print(Xtest.shape)
#     print(ytrain.shape)
#     print(ytest.shape)

#     Xtrain = Xtrain[:,imp]
#     Xtest = Xtest[:,imp]
    
    clf = LogisticRegression(max_iter=5000, C=C).fit(Xtrain, ytrain)
    yhat = clf.predict(Xtest)
    acc = sum(yhat == ytest)/len(ytest)
    
    print(f'{run}. {acc}')
    
    tot.append(acc)
    
#     continue
    
#     w = clf.coef_.squeeze()
#     w = np.mean(np.expand_dims(w,0)*Xtest, axis=0)
    
    w = clf.coef_.squeeze()
#     wNGT = np.mean(np.expand_dims(w,0)*Xtest[:c], axis=0)
#     wT2D = np.mean(np.expand_dims(w,0)*Xtest[c:], axis=0)
#     w = (wT2D+wNGT)/2
    
    jsonCompare = desc
    jsonAccuracy = acc
    jsonTrain = [a,b]
    jsonTest = [c,d]
    jsonWeights = w
    jsonLabels = countNames2 #[int(idx) for idx in list(np.arange(Xtrain.shape[1]))]

    jsonObj = {
        'Compare': jsonCompare,
        'Model': 'Logistic Regression',
        'Accuracy': jsonAccuracy,
        'Train': jsonTrain,
        'Test': jsonTest,
        'Weights': list(jsonWeights),
        'Labels': jsonLabels
    }
    
    dat = JsonData(analysis)
    dat.dict.update(jsonObj)
    r = dat.post()
    print(r.content)
    
tot = np.array(tot)
print(np.mean(tot))
print(np.std(tot))

['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10']
['259-260', '259-261', '259-262', '259-263', '260-261', '260-262', '260-263', '261-262', '261-263', '262-263']
0. 0.7777777777777778
b'Success'
1. 0.6666666666666666
b'Success'
2. 0.7777777777777778
b'Success'
3. 0.7777777777777778
b'Success'
4. 0.7777777777777778
b'Success'
5. 0.5555555555555556
b'Success'
6. 0.8888888888888888
b'Success'
7. 0.5555555555555556
b'Success'
8. 0.6666666666666666
b'Success'
9. 0.8888888888888888
b'Success'
10. 0.4444444444444444
b'Success'
11. 0.6666666666666666
b'Success'
12. 0.5555555555555556
b'Success'
13. 0.6666666666666666
b'Success'
14. 0.4444444444444444
b'Success'
15. 0.6666666666666666
b'Success'
16. 0.7777777777777778
b'Success'
17. 0.5555555555555556
b'Success'
18. 0.7777777777777778
b'Success'
19. 0.5555555555555556
b'Success'
20. 0.5555555555555556
b'Success'
21. 0.6666666666666666
b'Success'
22. 0.6666666666666666
b'Success'
23. 0.4444444444444444
b'Success'
24. 0.66666

In [26]:
(a,b,c,d)

(12, 15, 4, 5)

In [70]:
ngt, t2d = getSamples(dataMap, timep)
ngt = np.stack(ngt)
t2d = np.stack(t2d)

X = np.concatenate([ngt, t2d])
X = X.T

y = np.concatenate([np.ones((1,len(ngt))), -np.ones((1,len(t2d)))], axis=1)
y = np.repeat(y, 66023, axis=0)

print(y.shape)
print(X.shape)

(66023, 36)
(66023, 36)


In [71]:
sxy = np.einsum('ab,ab->a',X,y)
sxx = np.einsum('ab,ab->a',X,X)
syy = np.einsum('ab,ab->a',y,y)

rho = sxy/np.sqrt(sxx*syy)

print(rho.shape)
print(np.sum(np.isnan(rho)))

rho[np.isnan(rho)] = 0

(66023,)
28437


/tmp/ipykernel_88085/1672266886.py:5: RuntimeWarning: invalid value encountered in divide
  rho = sxy/np.sqrt(sxx*syy)
